In [2]:
import os
import json
import pandas as pd

import mlflow

import requests
from flask import Flask, request, jsonify

from pymongo import MongoClient

PUBLIC_SERVER_IP = os.getenv("PUBLIC_SERVER_IP")
# EVIDENTLY_SERVICE_ADDRESS = os.getenv('EVIDENTLY_SERVICE', 'http://127.0.0.1:6000') #Original was 5000 is used by MLFlow
# MONGODB_ADDRESS = os.getenv("MONGODB_ADDRESS", "mongodb://127.0.0.1:27017")

print(f"... Connecting to MLFlow Server on {PUBLIC_SERVER_IP} ...")

# MLFLOW_TRACKING_URI = 'sqlite:///../mlops-project.db'
MLFLOW_TRACKING_URI = f"http://{PUBLIC_SERVER_IP}:5000"
# mlflow.set_tracking_uri("http://127.0.0.1:5000/")
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

mongo_client = MongoClient(MONGODB_ADDRESS)
db = mongo_client.get_database("prediction_service")
collection = db.get_collection("data")

model_name = "Auction-car-prices-prediction"
model_uri = f"models:/{model_name}/production"

print("... Loading prediction model from production stage ...")

model = mlflow.pyfunc.load_model(model_uri=model_uri)

def prediction(record):
    record_df = pd.DataFrame([record])
    price_prediction = model.predict(record_df)

    return float(price_prediction[0])

def save_to_db(record, prediction):
    rec = record.copy()
    rec['prediction'] = prediction
    collection.insert_one(rec)


def send_to_evidently_service(record, prediction):
    rec = record.copy()
    rec['prediction'] = prediction
    requests.post(f"{EVIDENTLY_SERVICE_ADDRESS}/iterate/taxi", json=[rec])

print("... Ready to requests ...")

app = Flask("Car-price-prediction-service")

@app.route('/prediction', methods = ['POST'])
def prediction_endpoint():
    
    record = request.get_json()
    print("\nEstimating of the car with following characteristics\n", json.dumps(record, indent=4))
    
    price_prediction = prediction(record)
    print(price_prediction)
    result = {
        "price_estimation": price_prediction
    }
    
    save_to_db(record, float(price_prediction))
    send_to_evidently_service(record, float(price_prediction))
    return jsonify(result)

# if __name__ == "__main__":
#     app.run(debug=True, host='0.0.0.0', port=9696)



... Connecting to MLFlow Server on 51.250.101.205 ...
... Loading prediction model from production stage ...


 * Tip: There are .env or .flaskenv files present. Do "pip install python-dotenv" to use them.


... Ready to requests ...
 * Serving Flask app 'Car-price-prediction-service' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on all addresses.
 * Running on http://10.129.0.20:9696/ (Press CTRL+C to quit)
 * Restarting with stat
/homw/winx/.conda/envs/mlops-project-monitoring/lib/python3.9/site-packages/traitlets/traitlets.py:2202: FutureWarning: Supporting extra quotes around strings is deprecated in traitlets 5.0. You can use 'hmac-sha256' instead of '"hmac-sha256"' if you require traitlets >=5.
  warn(
/homw/winx/.conda/envs/mlops-project-monitoring/lib/python3.9/site-packages/traitlets/traitlets.py:2157: FutureWarning: Supporting extra quotes around Bytes is deprecated in traitlets 5.0. Use 'fc017bc5-f5a8-4f3c-a4cd-046a69fd36eb' instead of 'b"fc017bc5-f5a8-4f3c-a4cd-046a69fd36eb"'.
  warn(
Traceback (most recent call last):
  File "/homw/winx/.conda/envs/mlops-project-monitoring/lib/python3.9/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/homw/winx/.conda/envs/mlops-project-monitoring/lib/python3.9/site-packages/traitlets/config/application.py

SystemExit: 1

/homw/winx/.conda/envs/mlops-project-monitoring/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3406: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


: 

In [3]:
PUBLIC_SERVER_IP = os.getenv("PUBLIC_SERVER_IP")
print(PUBLIC_SERVER_IP)

51.250.101.205


In [1]:
%env PUBLIC_SERVER_IP=51.250.101.205

env: PUBLIC_SERVER_IP=51.250.101.205
